In [72]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import cv2
from glob import glob
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.models as models
from torch.autograd import Variable
import torchvision.transforms as transforms
from PIL import Image, ImageFile
from torch.utils.data import TensorDataset, DataLoader,Dataset
from sklearn.model_selection import train_test_split
import copy 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
ImageFile.LOAD_TRUNCATED_IMAGES = True
#print(os.listdir("../input"))
#../input/train/train
#../input/train/test

# Any results you write to the current directory are saved as output.
use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('No GPU found. Please use a GPU to train your neural network.')

In [73]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('sample_submission.csv')

In [74]:
df_train = df_train[:18528]
print(df_train.shape)
df_train.head()

(18528, 2)


,image_id,category
0,0,77
1,1,81
2,2,52
3,3,72
4,4,58


In [75]:
# NOTE: class is inherited from Dataset
class ImageLabelDataset(Dataset):
    def __init__(self, df_data, prediction, folder="train"):
        super().__init__()
        self.df = df_data.values
        self.prediction = prediction.values
        self.folder = folder

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        tensorimage = self.preprocess_image(self.df[index])
        label = self.prediction[index]
        label_tensor = self.get_dummies(label)
        #label_tensor = label
        #x = np.squeeze(label_tensor.detach().cpu().numpy())
        #print(x.argsort()[-6:][::-1])
        #print(len(label_tensor))
        #print(label_tensor.shape)
        return tensorimage, label_tensor
    
    def preprocess_image(self, img_path):
        data_transform = transforms.Compose([transforms.ToPILImage(),
                                             transforms.Resize(299), 
                                             transforms.RandomResizedCrop(299), 
                                             transforms.RandomHorizontalFlip(),
                                             transforms.RandomVerticalFlip(),
                                             transforms.RandomRotation(degrees=30), 
                                             transforms.ToTensor(),
                                             transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                            ])
        image = cv2.imread("{}/{}.jpg".format(self.folder, img_path))
        image = data_transform(image)
        return image
    
    def get_dummies(self, attribute_id):
        label_tensor = torch.zeros((1, 103))
        #for label in attribute_id.split():
        label_tensor[0, int(attribute_id)-1] = 1
        return label_tensor

In [76]:
# df_train.head()
batch_size = 32
train_image = df_train["image_id"]
#target = df_train.drop(['id', 'attribute_ids'],axis=1)
target = df_train["category"]
X_train, X_val, y_train, y_val = train_test_split(train_image, target, random_state=42, test_size=0.1)

test_image = df_test["image_id"]
test_target = df_test["category"]
#test_target = df_test.drop(['id', 'attribute_ids'],axis=1)

train_set = ImageLabelDataset(df_data=X_train, prediction=y_train, folder="train")
val_set = ImageLabelDataset(df_data=X_val, prediction=y_val, folder="train")
predict_set = ImageLabelDataset(df_data=test_image, prediction=test_target, folder="test")

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=1)
test_val_loader = torch.utils.data.DataLoader(val_set, batch_size=1, shuffle=False, num_workers=1)
test_loader = torch.utils.data.DataLoader(predict_set, batch_size=1, num_workers=0)


In [117]:
# Hyperparameters
n_output=1
# Number of Epochs
num_epochs = 10
# Learning Rate
learning_rate = 0.0001
# Model parameters

# Show stats for every n number of batches
show_every_n_batches = 1

In [83]:
def train_rnn(model, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    train_loss = 0
    valid_loss = 0
    batch_losses = []
    val_batch_losses = []
    valid_loss_min = np.Inf
    
    model.train()
    
    previousLoss = np.Inf
    minLoss = np.Inf

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            #print(batch_idx + len(train_loader.dataset))
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            target = np.squeeze(target)
            #target = target.view(1,-1)
            target= target.long()
            #print(np.squeeze(target))
            #targs = target.view(-1).long()
            # print(np.squeeze(target))
            #print(torch.max(target, 1)[1])
            #print(data)
            optimizer.zero_grad()
            # forward pass: Train compute predicted outputs by passing inputs to the model
            output = model.forward(data)
            output, _ = output
            #print(output.squeeze(1))
            #print(type(output))
            #_, preds = torch.max(output.data, 1)
            #print(torch.max(target, 1)[1])
            #print("output")
            #print(torch.max(output, 1)[1])
            # calculate the Train batch loss
            #output = Variable(output.float())
            #target = Variable(target.long())
            loss = criterion(output, torch.max(target, 1)[1])
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            #print(loss)
            # update training loss
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            #train_loss += ((1 / (batch_idx + 1)) * (loss.item() - train_loss))            
            train_loss += loss.item()
            #print(loss.item())
            
        model.eval()
        for batch_idx, (data, target) in enumerate(val_loader):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            # forward pass: compute predicted outputs by passing inputs to the model
            target = np.squeeze(target)
            target= target.long()
            #target = target.view(1,-1)
            
            output = model.forward(data)
            #print(output)
            #print(target)
            #output, _ = output
            # calculate the Val batch loss
            loss = criterion(output, torch.max(target, 1)[1])
            # update average validation loss 
            #val_batch_losses.append(loss.item())
            #valid_loss += ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            valid_loss += loss.item()
            #print(loss.item())
        
        valid_loss = valid_loss/len(val_loader.dataset)
        train_loss = train_loss/len(train_loader.dataset)
        
        # print training/validation statistics 
        if epoch_i%show_every_n_batches == 0:
            print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch_i, train_loss, valid_loss))
        
            ## TODO: save the model if validation loss has decreased
            # save model if validation loss has decreased
            if valid_loss < valid_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
                with open('trained_rnn_new', 'wb') as pickle_file:
                    torch.save(model.state_dict(), 'save/trained_rnn_new')
                valid_loss_min = valid_loss
                train_loss = 0
                valid_loss = 0
                #batch_losses = []
                #val_batch_losses = []
  
    return model

In [79]:
model_transfer = models.inception_v3(pretrained=True)
print(model_transfer)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [118]:
# Freeze training for all "features" layers
#for param in model_transfer.features.parameters():
    #param.requires_grad = False

## Freezing all layers
for params in model_transfer.parameters():
    params.requires_grad = False

## Freezing the first few layers. Here I am freezing the first 7 layers 
        
custom_model = nn.Sequential(nn.Linear(2048, 1024), 
                  nn.ReLU(),
                  nn.Dropout(p=0.5), 
                  nn.Linear(1024, 102)
                 )

model_transfer.fc = custom_model
#model_transfer.classifier = custom_model

model_transfer.load_state_dict(torch.load('save/trained_rnn_new'))

if use_cuda:
    model_transfer = model_transfer.cuda()

# print(model_transfer)

# specify loss function
criterion_scratch = nn.CrossEntropyLoss()
#criterion_scratch = nn.BCELoss()
#criterion_scratch = nn.BCELoss(reduction="mean").to('cuda:0')

# specify optimizer
optimizer_scratch = optim.SGD(model_transfer.parameters(), lr=learning_rate, momentum=0.9)
#optimizer_scratch = optim.Adam(model_transfer.classifier.parameters(), lr=learning_rate)
#optimizer_scratch = optim.SGD(list(filter(lambda p: p.requires_grad, model_transfer.parameters())), lr=learning_rate, momentum=0.9)


trained_rnn = train_rnn(model_transfer, batch_size, optimizer_scratch, criterion_scratch, num_epochs, show_every_n_batches)

Training for 10 epoch(s)...
Epoch: 1 	Training Loss: 0.048963 	Validation Loss: 0.035549
Validation loss decreased (inf --> 0.035549).  Saving model ...
Epoch: 2 	Training Loss: 0.048180 	Validation Loss: 0.035746
Epoch: 3 	Training Loss: 0.048594 	Validation Loss: 0.035054
Validation loss decreased (0.035549 --> 0.035054).  Saving model ...
Epoch: 4 	Training Loss: 0.048142 	Validation Loss: 0.035022
Validation loss decreased (0.035054 --> 0.035022).  Saving model ...
Epoch: 5 	Training Loss: 0.048809 	Validation Loss: 0.032463
Validation loss decreased (0.035022 --> 0.032463).  Saving model ...
Epoch: 6 	Training Loss: 0.047482 	Validation Loss: 0.034268
Epoch: 7 	Training Loss: 0.048381 	Validation Loss: 0.034375
Epoch: 8 	Training Loss: 0.048544 	Validation Loss: 0.033418
Epoch: 9 	Training Loss: 0.047943 	Validation Loss: 0.035477
Epoch: 10 	Training Loss: 0.047770 	Validation Loss: 0.035886


In [119]:
model_transfer.load_state_dict(torch.load('save/trained_rnn_new'))
model_transfer.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [123]:
from sklearn.metrics import r2_score

preds = []
targetpred = []
for batch_idx, (data, target) in enumerate(test_val_loader):
    # move to GPU
    #print(np.squeeze(target.detach().cpu().numpy()))
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    ## update the average validation loss
    # forward pass: compute predicted outputs by passing inputs to the model
    target = np.squeeze(target)
    #target= target.long()
    output = model_transfer.forward(data)    
    #print(target)
    #print(torch.max(target, 0))
    #print(output)
    #print(torch.max(output, 1)[1].detach().cpu().numpy())
    #values, indices = output.max(1)    
    pr = torch.max(output, 1)[1].detach().cpu().numpy()
    tpr = torch.max(target, 0)[1].detach().cpu().numpy()
    #print(tpr)
    preds.append(int(pr[0]) + 1)
    targetpred.append(int(tpr))
    #print(pr.detach().cpu())
print(preds[:100])
print(targetpred[:100])
r2_score(targetpred, preds)

[57, 80, 47, 12, 30, 73, 26, 51, 72, 94, 76, 57, 81, 43, 46, 58, 46, 94, 72, 77, 8, 98, 81, 98, 24, 42, 76, 15, 11, 95, 37, 37, 83, 73, 73, 19, 5, 75, 2, 91, 76, 8, 77, 51, 56, 89, 16, 28, 77, 55, 41, 83, 31, 73, 73, 44, 49, 60, 70, 52, 83, 89, 56, 63, 26, 46, 13, 75, 77, 78, 19, 43, 8, 52, 88, 25, 44, 94, 71, 84, 56, 78, 5, 48, 98, 8, 11, 44, 36, 50, 76, 79, 8, 65, 43, 58, 12, 72, 14, 5]
[56, 79, 73, 11, 29, 72, 25, 50, 71, 93, 54, 56, 94, 42, 10, 52, 14, 93, 74, 76, 7, 97, 80, 69, 49, 41, 75, 41, 10, 71, 36, 36, 82, 80, 72, 18, 4, 74, 1, 90, 75, 7, 76, 75, 95, 88, 15, 27, 76, 54, 40, 82, 30, 77, 6, 43, 48, 59, 69, 51, 82, 88, 55, 62, 25, 45, 12, 74, 76, 86, 18, 23, 7, 51, 87, 24, 43, 93, 70, 83, 55, 77, 4, 15, 97, 7, 50, 43, 35, 49, 75, 78, 16, 41, 52, 57, 11, 71, 13, 26]


0.4648664587566038

In [124]:
preds = []
np.set_printoptions(threshold=100)
submission = pd.read_csv('sample_submission.csv')
for batch_idx, (data, target) in enumerate(test_loader):
    # move to GPU
    #print(np.squeeze(target.detach().cpu().numpy()))
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    ## update the average validation loss
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_transfer.forward(data)
    #print(target)
    #print(output)
    #print(torch.max(output, 1)[1].detach().cpu().numpy())
    #values, indices = output.max(1)    
    pr = torch.max(output, 1)[1].detach().cpu().numpy()
    preds.append(int(pr[0]) + 1)
    #for i in pr:
        #preds.append(i)
    #print(batch_idx)
    #print(pr)
    #print(x.argsort()[-6:][::-1])
    #print(pr.argsort()[-6:][::-1])
    #print(values)
    # calculate the batch loss
submission["category"] = preds
print(submission.head())


   image_id  category
0     18540         9
1     18541        40
2     18542        78
3     18543        72
4     18544        44


In [125]:
submission.to_csv('submission.csv', index=False)